#### -----
#### Build a Large Language Model
#### Sebastian Raschka
#### -----

# Coding attention mechanisms

  - This chapter will focus on coding the remaining parts of the LLMs surrounding the self-attention mechanism to see it in action and to create a model to generate text.
  - Different types of attention mechanisms
    - Simplified self-attention:
    - Self-attention: Simplified self-attention with trainable weights
    - Casual attention: Type of self-attention used in LLMs that allows a model to consider on previous and current inputs in a sequence
    - Multi-head attention:

## 3.1 - The problem with modeling long sequences

  - You can't simply translate a text word into another language due to the grammatical structures in source and target language.
  - Deep neural network with two submodules, encoder and decoder:
    - encode : first read in and process the entire text
    - decoder : then produces the translated text.
  - Before transformers, recurrent neural networks (RNNs) were the most popular encoder-decoder architecture for language translation.

## 3.2 - Capturing data dependencies with attention mechanisms

  -